In [1]:
import numpy as np
def generate_non_linear_three_input_two_output(number_of_rows, dimensions, weights_0, bias_0, weights_1, bias_1):

  n = number_of_rows
  d = dimensions

  # Generate numbers between -1.0 and 1.0, n rows and 3 columns
  x = np.random.uniform(-1, 1, (n, d))

  # Showing 20 of them
  # print(x[:10])

  # y = w * x + b
  # y = (w_0 * x_0) + (w_1 * x_1) + (w_2 * x_2)

  # Randomly assigning weights
  weights_true_0 = np.array([weights_0]).T

  # print(weights_true)

  bias_true_0 = np.array(bias_0)
  # print(bias_true)

  # print(x.shape, weights_true.shape, bias_true.shape)

  # Matmult with data and wieghts, adding bias_true as well.
  y_true_0 = ((x ** 2) @ weights_true_0) + (x @ weights_true_0) + bias_true_0
  # print(y_true[:10])

  print (f'x: {x.shape}, weights: {weights_true_0.shape}, bias: {bias_true_0.shape}, y: {y_true_0.shape}')

  # Randomly assigning weights
  weights_true_1 = np.array([weights_1]).T

  # print(weights_true)

  bias_true_1 = np.array(bias_1)
  # print(bias_true)

  # print(x.shape, weights_true.shape, bias_true.shape)

  # Matmult with data and wieghts, adding bias_true as well.
  y_true_1 = ((x ** 2) @ weights_true_1) + (x @ weights_true_1) + bias_true_1
  # print(y_true[:10])

  print (f'x: {x.shape}, weights: {weights_true_1.shape}, bias: {bias_true_1.shape}, y: {y_true_1.shape}')

  return { "input" : x, "weights_true_0" : weights_true_0, "bias_true_0" : bias_true_0, "y_true_0" : y_true_0, "weights_true_1" : weights_true_1, "bias_true_1" : bias_true_1, "y_true_1" : y_true_1}

In [2]:
three_input_two_output_func = generate_non_linear_three_input_two_output(200, 3, [3, -2, 1], [.2], [2, -1, 3], [.4])

x: (200, 3), weights: (3, 1), bias: (1,), y: (200, 1)
x: (200, 3), weights: (3, 1), bias: (1,), y: (200, 1)


In [3]:
# Now we run the training loop
from typing import Any, Callable
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf

class Linear(keras.layers.Layer):
  """y = w.x + b"""

  def __init__(self, units=1):
      super(Linear, self).__init__()
      self.units = units

  def build(self, input_shape):
      self.input_size = input_shape[1]
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

  def call(self, inputs):
     
      return tf.matmul(inputs, self.w) + self.b



# Let's reuse the Linear class
# with a `build` method that we defined above.

class MLP(keras.layers.Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear()
        self.linear_2 = Linear()
        self.linear_3 = Linear()

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

class ActivityRegularization(keras.layers.Layer):
  """Layer that creates an activity sparsity regularization loss."""
  
  def __init__(self, rate=1e-2):
    super(ActivityRegularization, self).__init__()
    self.rate = rate
  
  def call(self, inputs):
    # We use `add_loss` to create a regularization loss
    # that depends on the inputs.
    self.add_loss(self.rate * tf.reduce_sum(inputs))
    return inputs

class SparseMLP(keras.layers.Layer):
  """Stack of Linear layers with a sparsity regularization loss."""

  def __init__(self):
      super(SparseMLP, self).__init__()
      self.linear_1 = Linear(1)
      self.regularization = ActivityRegularization(1e-2)
      self.linear_3 = Linear(1)

  def call(self, inputs):
      x = self.linear_1(inputs)
      x = tf.nn.relu(x)
      x = self.regularization(x)
      return self.linear_3(x)
    

In [4]:
# Now we run the training loop
from typing import Any, Callable
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf

def tensorfit():
  
  three_input_two_output_func = generate_non_linear_three_input_two_output(200, 3, [3, -2, 1], [.2], [2, -1, 3], [.4])
  x = tf.convert_to_tensor(three_input_two_output_func['input'])
  y_true_0_1 = [tf.convert_to_tensor(three_input_two_output_func['y_true_0']), tf.convert_to_tensor(three_input_two_output_func['y_true_1'])]
  d = 3
  
  lr = 0.1
  num_epochs = 40
  
  loss_0 = tf.keras.losses.MeanSquaredError()
  model_0 = SparseMLP()

  loss_1 = tf.keras.losses.MeanSquaredError()
  model_1 = SparseMLP()

  optimizer_0 = tf.keras.optimizers.SGD(learning_rate=lr)
  optimizer_1 = tf.keras.optimizers.SGD(learning_rate=lr)

  for epoch in range(num_epochs):
    
    with tf.GradientTape() as tape:
      y_pred_tensor_0 = model_0(x)
      loss_value_0 = loss_0(y_pred_tensor_0, y_true_0_1[0])
      loss_value_0 = tf.cast(loss_value_0, tf.float32, name=None)
      loss_value_0 += sum(model_0.losses)
      gradients_0 = tape.gradient(loss_value_0, model_0.trainable_weights)
      optimizer_0.apply_gradients(zip(gradients_0, model_0.trainable_weights))

    with tf.GradientTape() as tape:
      y_pred_tensor_1 = model_1(x)
      loss_value_1 = loss_1(y_pred_tensor_1, y_true_0_1[1])
      loss_value_1 = tf.cast(loss_value_1, tf.float32, name=None)
      loss_value_1 += sum(model_1.losses)
      gradients_1 = tape.gradient(loss_value_1, model_1.trainable_weights)
      optimizer_1.apply_gradients(zip(gradients_1, model_1.trainable_weights))      
      
    print(loss_value_0, loss_value_1)


tensorfit()


x: (200, 3), weights: (3, 1), bias: (1,), y: (200, 1)
x: (200, 3), weights: (3, 1), bias: (1,), y: (200, 1)
tf.Tensor(5.888475, shape=(), dtype=float32) tf.Tensor(8.5227585, shape=(), dtype=float32)
tf.Tensor(5.586216, shape=(), dtype=float32) tf.Tensor(7.3731966, shape=(), dtype=float32)
tf.Tensor(5.425371, shape=(), dtype=float32) tf.Tensor(6.716572, shape=(), dtype=float32)
tf.Tensor(5.3224306, shape=(), dtype=float32) tf.Tensor(6.296332, shape=(), dtype=float32)
tf.Tensor(5.256548, shape=(), dtype=float32) tf.Tensor(6.027378, shape=(), dtype=float32)
tf.Tensor(5.2143836, shape=(), dtype=float32) tf.Tensor(5.855248, shape=(), dtype=float32)
tf.Tensor(5.187398, shape=(), dtype=float32) tf.Tensor(5.745084, shape=(), dtype=float32)
tf.Tensor(5.1701274, shape=(), dtype=float32) tf.Tensor(5.67458, shape=(), dtype=float32)
tf.Tensor(5.159075, shape=(), dtype=float32) tf.Tensor(5.629457, shape=(), dtype=float32)
tf.Tensor(5.152001, shape=(), dtype=float32) tf.Tensor(5.600578, shape=(), dty